#### 训练数据预处理
1. 导入训练数据

In [15]:
import pandas as pd
import numpy as np

# 这里的编码类型可以用sublime打开查看左下角
base_test = pd.read_csv('tables/base-test.csv',engine='python',encoding='utf8')
knowledge_test = pd.read_csv('tables/knowledge-test.csv',engine='python',encoding='utf8')
money_test = pd.read_csv('tables/money-test.csv',engine='python',encoding='gb2312')
year_test = pd.read_csv('tables/year-test.csv',engine='python',encoding='gb2312')

2. 处理base_test数据

    2.1 讲中文属性编码

    2.2 去除无关信息

    2.3 缺失值处理（均值）

In [16]:
# 定义map字典
map_industry = {'零售业':1,'服务业':2,'工业':3,'商业服务业':4,'社区服务':5,'交通运输业':6}
map_type_enterprise = {'有限责任公司':10,'合伙企业':20,'股份有限公司':30,'农民专业合作社':40,'集体所有制企业':50}
map_type_controller = {'自然人':10,'企业法人':20}
# 2.1 给中文属性编码
base_test['行业'] = base_test['行业'].map(map_industry)
base_test['企业类型'] = base_test['企业类型'].map(map_type_enterprise)
base_test['控制人类型'] = base_test['控制人类型'].map(map_type_controller)
# 2.2 去除无关信息
base_test_data = base_test.drop(columns=['区域'])
# 2.3 缺失值补全（平均值）
for column in list(base_test_data.columns[base_test_data.isnull().sum() > 0]):
    a = base_test_data[column].mean()
    base_test_data[column].fillna(a,inplace=True)

3. 处理knowledge_test数据

    3.1 缺失值处理(01数据类型，直接round四舍五入即可)


In [17]:
# 3.1 缺失值处理
for column in list(knowledge_test.columns[knowledge_test.isnull().sum() > 0]):
    a = round(knowledge_test[column].mean())
    knowledge_test[column].fillna(a,inplace=True)


4. 处理money_test和year_test数据

    4.1 根据ID和year两个属性合并两个数据集

    4.2 提取15、16和17年的数据

    4.3 将15、16和17的数据合并成一张表

In [18]:
# 4.1 根据ID和year两个属性合并两个数据集
money_year_test = pd.merge(money_test,year_test,on=['ID','year'])
# 4.2 提取15、16和17年的数据
money_year_test_15 = money_year_test.loc[money_year_test['year']==2015].add_suffix('_15')
money_year_test_15.rename(columns={'ID_15':'ID','year_15':'year'},inplace=True)
money_year_test_16 = money_year_test.loc[money_year_test['year']==2016].add_suffix('_16')
money_year_test_16.rename(columns={'ID_16':'ID','year_16':'year'},inplace=True)
# 4.3 合并三年的数据
money_year_test_1516=pd.merge(money_year_test_15,money_year_test_16,on='ID')

5. 整合所有的表

    5.1 将base和knowledge表连接起来(采用内连接)

    5.2 将所有表连接

    5.3 去除多余的属性（year_x,year_y,year）

    5.4 填充缺失值（平均值）

In [19]:
# 5.1 将base和knowledge表连接起来(采用内连接)
base_knowledge_test = pd.merge(base_test_data,knowledge_test,on='ID',how='inner')
# 5.2 将所有表连接
test_data = pd.merge(money_year_test_1516,base_knowledge_test,on='ID')
# 5.3 去除多余的属性（year_x,year_y,year）
test_data = test_data.drop(columns=['year_x','year_y'])
# 5.4 填充缺失值（平均值）
for column in list(test_data.columns[test_data.isnull().sum() > 0]):
    a = int(test_data[column].mean())
    test_data[column].fillna(a,inplace=True)
# 最终的训练数据
test_data.to_csv('tables/test.csv')